In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import re
import seaborn as sns
import time

from collections import Counter, defaultdict
from matplotlib import rcParams
from sklearn.linear_model import LogisticRegression

%matplotlib inline

In [ ]:
def get_upper_bound(num_tests = 1, alpha = 0.05, one_tailed = False):
    """Helper function for implementing bootstrap percentile method"""
    lower = alpha/num_tests/2
    if one_tailed:
        lower *= 2
    return 1 - lower


def get_upper_index(upper_bound, distribution):
    """Find index based on upper bound and distribution size"""
    upper_idx = int(np.floor(upper_bound * len(distribution)))
    return upper_idx


def tjur_r2_ALL(X_df, y_df):
    """
    Tjur, T. (2009). Coefficients of determination in logistic regression 
    models—a new proposal: The coefficient of discrimination. The American 
    Statistician, 63(4),366-372. DOI: 10.1198/tast.2009.08210
    """
    count = 0
    r2s = []
    y_cols = y_df.columns
    X_cols = X_df.columns
    X = X_df[X_cols].to_numpy()
    for y_col in y_cols:
        y = y_df[y_col].values
        r2_row = []
        for x_col in X_cols:
            X = X_df[x_col].values.reshape(-1,1)
            logit = LogisticRegression(penalty='none', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, max_iter=2000, multi_class='auto', verbose=0).fit(X, y)
            yhat = logit.predict_proba(X)
            yhat = [yh[1] for yh in yhat]
            response = pd.DataFrame(list(zip(y, yhat)), columns=["y", "yhat"])
            r2 = np.mean(response[response["y"]==True]["yhat"]) - np.mean(response[response["y"]==False]["yhat"])
            r2_row.append(r2)
            count += 1
        X = X_df[X_cols].to_numpy()
        logit = LogisticRegression(penalty='none', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, max_iter=2000, multi_class='auto', verbose=0).fit(X, y)
        yhat = logit.predict_proba(X)
        yhat = [yh[1] for yh in yhat]
        response = pd.DataFrame(list(zip(y, yhat)), columns=["y", "yhat"])
        r2_full = np.mean(response[response["y"]==True]["yhat"]) - np.mean(response[response["y"]==False]["yhat"])
        r2_row.append(r2_full)
        r2s.append(r2_row)
    return np.array(r2s)


def hypothesis_testing_r2(real_ests, sim_ests_dict, alpha = 0.05, bonferroni = True, one_tailed = False):
    """
    Note: Relaxed assumption that test should be against simulated values in cell_ij only.
    Tests are against all Tjur R2s for code_i: increased by a factor of k.
    Excludes last value (estimate from regressing code on full topic model) for pairwise
    hypothesis tests.
    """
    num_codes, num_cols = real_ests.shape
    test_mat = np.zeros((num_codes, num_cols))
    num_topics = num_cols - 1
    num_tests = 1
    if bonferroni == True:
        num_tests = real_ests.size
    upper_bound = get_upper_bound(num_tests, alpha, one_tailed)
    # pairwise tests (excluding last columns):
    for i, row in enumerate(real_ests):
        sims = [k15_dict[i][k] for k in range(15)] # pairwise, excludes last column
        sims = sorted([sim for cell in sims for sim in cell])
        upper_idx = get_upper_index(upper_bound, sims)
        test_stat = sims[upper_idx]
        for j, col in enumerate(row[:-1]): # pairwise, excluding last column
            real_r2 = real_ests[i][j]
            if real_r2 > test_stat:
                test_mat[i][j] = 1
    print(np.count_nonzero(test_mat)/test_mat.size)
    # tests on full models:
    for i, row in enumerate(real_ests):
        full_model_r2 = real_ests[i][-1]
        last_key = sorted(list(sim_ests_dict[i]))[-1]
        sims = sorted(sim_ests_dict[i][last_key]) # last column, 1000 simulations
        assert len(sims) == 1000
        upper_idx = get_upper_index(upper_bound, sims)
        test_stat = sims[upper_idx]
        if full_model_r2 > test_stat:
            test_mat[i][-1] = 1
    print(np.count_nonzero(test_mat)/test_mat.size)
    return test_mat


def filter_columns(sig_test_mat_):
    """Identify columns to exclude if filter_cols == False"""
    idx_to_filter = []
    num_cols = sig_test_mat_.shape[1]
    for i in range(num_cols):
        col = sig_test_mat_[:, i]
        sum_ = sum(col)
        if sum_ == 0:
            idx_to_filter.append(i)
    return idx_to_filter
    

def results_to_strings(results, sig_tests, filter_cols=False):
    """Add asterisks to significant results (etc.)"""
    mat = []
    for i, row in enumerate(results):
        row_str = []
        for j, cell in enumerate(row):
            res = f"{cell:.3f}"
            if sig_tests[i][j]:
                res += "*"
            row_str.append(res)
        mat.append(row_str)
    mat = np.array(mat)
    col_names = [f"Topic {i+1}" for i in range(sig_tests.shape[1])]
    if filter_cols:
        idx_to_filter = filter_columns(sig_tests)
        mat = np.delete(mat, idx_to_filter, 1)
        col_names = list(np.delete(col_names, idx_to_filter))
    return mat, col_names


def remove_star(string_: str) -> float:
    """Convert output of results_to_strings() to floats"""
    return float(string_.replace("*", ""))

In [ ]:
k15_dict = pickle.load(open("simulations_tjur_r2_k15.d", "rb"))
k35_dict = pickle.load(open("simulations_tjur_r2_k35.d", "rb"))

In [ ]:
len(k15_dict[0][0])

In [ ]:
df_k15 = pd.read_csv("df_for_comparison_k15.csv")
df_k35 = pd.read_csv("df_for_comparison_k35.csv")

topic_cols = [f"topic_{i}" for i in range(1,36)]

df_k15 = df_k15[topic_cols[:15]]
df_k35 = df_k35[topic_cols[:35]]

df_qual = pd.read_csv("df_for_comparison_k15.csv")
cols = list(df_qual.columns)[13:-18]
cols.remove("Index Codes Applied")

In [ ]:
cols

In [ ]:
df_qual.columns

In [ ]:
orig_cols = ['Academic Medicine Applied',
 'Culture of medicine Applied',
 'Expectations Applied',
 'Hospital/clinic hours and environment Applied',
 'Incentive/payment structure Applied',
 'Interpersonal Applied',
 'Job changes Applied',
 'Medical training Applied',
 'Missed opportunities Applied',
 'Pay/Compensation Applied',
 'Psychological Applied',
 'Sub-specialities Applied',
 'Great quote/example Applied',
 'Motherhood Specific Applied',
 'Motherhood Specific/Breastfeeding/Pumping Applied',
 'Motherhood Specific/Childcare/Household challenges Applied',
 'Motherhood Specific/Family leave Applied']

assert cols == orig_cols

qual_codes = [code.replace(" Applied", "") for code in cols]

df_qual = df_qual[cols]

display(df_qual.head())

display(df_k15.head())

display(df_k35.head())

In [ ]:
print(df_qual.shape)
print(df_k15.shape)
print(df_k35.shape)

In [ ]:
print(set(map(lambda x: round(x, 6), df_k15.sum(axis=1)))) # checking that probabilities sum to approximately 1
print(set(map(lambda x: round(x, 6), df_k35.sum(axis=1)))) # checking that probabilities sum to approximately 1

In [ ]:
%%time

tjur_k15 = tjur_r2_ALL(df_k15, df_qual)
tjur_k35 = tjur_r2_ALL(df_k35, df_qual)

In [ ]:
tjur_k15.shape, tjur_k35.shape

In [ ]:
assert tjur_k15.shape[0] == len(k15_dict.keys())
assert tjur_k15.shape[1] == len(k15_dict[0].keys())
assert tjur_k35.shape[0] == len(k35_dict.keys())
assert tjur_k35.shape[1] == len(k35_dict[0].keys())

In [ ]:
tjur_k15.shape[0] * tjur_k15.shape[1]

In [ ]:
num_tests = 1
interval = 0.95
interval = 1 - (1 - interval)/num_tests
lower = (1 - interval)/2 * 100
upper = 100 - lower
print(f"95% confidence level without correction for multiple testing: {interval:.2f}")
print(f"Percentile of upper bound: {upper:.1f}", "\n")

num_tests = tjur_k15.shape[0] * tjur_k15.shape[1]
interval = 0.95
interval = 1 - (1 - interval)/num_tests
lower = (1 - interval)/2 * 100
upper = 100 - lower
print(f"95% confidence level with correction for multiple testing ({num_tests} tests): {interval:.6f}")
print(f"Percentile of upper bound: {upper:.5f}", "\n")

In [ ]:
k15_dict.keys() # indices for the 17 qualitative codes

In [ ]:
len(k15_dict[0]) # estimates for regressing code 0 on each of the 15 topics and then on all topics

In [ ]:
len(k15_dict[0][0]) # estimates assuming r2_ij should be compared to cell_ij in results of simulations

In [ ]:
tjur_k15.shape

## Figure 3. Coefficients of discrimination between codes and topics in preferred model (k = 15).

In [ ]:
topic_labels = ["Transitions", "Family Leave", "Promotion Inequality", "Power", "Burnout", "Unequal Compensation", 
                "Psychosocial Support", "Respect", "Training", "Staff Interactions", "Career Advancement", 
                "Favoritism", "Hierarchy", "Agency", "Pumping", "Full Model"]

sig_tests15 = hypothesis_testing_r2(tjur_k15, k15_dict, alpha = 0.05, bonferroni = True, one_tailed = False)
mat, _ = results_to_strings(tjur_k15, sig_tests15)
df = pd.DataFrame(mat, columns=topic_labels)
df.insert(0, "Code", qual_codes)

outf = "pref_model_table.csv"
df.to_csv(outf)

mask = np.invert(sig_tests15.astype(bool))

sns.set_style("white")

f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.color_palette("Blues", n_colors=1000)
rcParams['figure.figsize'] = 12,9
rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.family"] = "Times New Roman"

plot = sns.heatmap(tjur_k15, vmin=0.0, vmax=1.0, center=0.4,
            square=False, linewidths=0.01,
            linecolor="lightgray",
            yticklabels=qual_codes,
            xticklabels=topic_labels,
            annot=True,
            fmt=".3f",
            cmap = cmap,
            mask = mask)

plot.set_xticklabels(plot.get_xticklabels(), rotation=45, horizontalalignment="right")
plt.tight_layout()
for i in range(tjur_k15.shape[1]+1):
    plt.axvline(x=i, color="lightgray")
for i in range(18): # ?
    plt.axhline(y=i, color="lightgray")

plt.savefig("comparing_topic_models_qual_coding_figure3.png", format="png", transparent=False, dpi=600)

plt.show()

print("Figure 3. Coefficients of discrimination between codes and topics in preferred model (k = 15).")

## Figure 6. Coefficients of discrimination between codes and topics in alternative model (k = 35).

In [ ]:
topic_labels = ["Transitions", "Family Leave", "Promotion Inequality", "Power", "Burnout", "Unequal Compensation", 
                "Psychosocial Support", "Respect", "Training", "Staff Interactions", "Career Advancement", 
                "Favoritism", "Hierarchy", "Agency", "Pumping", "Full Model"]

sig_tests35 = hypothesis_testing_r2(tjur_k35, k35_dict, alpha = 0.05, bonferroni = True, one_tailed = False)
mat, _ = results_to_strings(tjur_k35, sig_tests35)
df = pd.DataFrame(mat, columns=[f"Topic {i}" for i in range(1,36)] + ["Full Model"])
df.insert(0, "Code", qual_codes)

outf = "alt_model_table.csv"
df.to_csv(outf)

mask = np.invert(sig_tests35.astype(bool))

sns.set_style("white")

f, ax = plt.subplots(figsize=(20, 9))

cmap = sns.color_palette("Blues", n_colors=1000)
rcParams['figure.figsize'] = 16,9
rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.family"] = "Times New Roman"

plot = sns.heatmap(tjur_k35, vmin=0.0, vmax=1.0, center=0.4,
            square=False, linewidths=0.01,
            linecolor="lightgray",
            yticklabels=qual_codes,
            xticklabels=[f"Topic {i}" for i in range(1,36)] + ["Full Model"],
            annot=True,
            fmt=".3f",
            cmap = cmap,
            mask = mask)

plot.set_xticklabels(plot.get_xticklabels(), rotation=45, horizontalalignment="right")
plt.tight_layout()
for i in range(tjur_k35.shape[1]+1):
    plt.axvline(x=i, color="lightgray")
for i in range(18): # ?
    plt.axhline(y=i, color="lightgray")

plt.savefig("comparing_topic_models_qual_coding_figure6.png", format="png", transparent=False, dpi=600)
plt.show()

print("Figure 6. Coefficients of discrimination between codes and topics in alternative model (k = 35).")

## Table 3. Explanation of variance in qualitative codes by preferred (k = 15) and alternative (k = 35) LDA topic models.

In [ ]:
pref_model = pd.read_csv("pref_model_table.csv")
alt_model = pd.read_csv("alt_model_table.csv")
pref = pref_model[["Code", "Full Model"]].rename(columns={"Full Model": "Preferred Model (Tjur R2)"})
alt = alt_model[["Code", "Full Model"]].rename(columns={"Full Model": "Alternative Model (Tjur R2)"})

comp_tab = pref.merge(alt, on = "Code")
comp_tab["R2 Ratio"] = comp_tab.apply(lambda row: remove_star(row["Preferred Model (Tjur R2)"])/remove_star(row["Alternative Model (Tjur R2)"]), axis = 1)
comp_tab["R2 Ratio"] = comp_tab["R2 Ratio"].apply(lambda x: f"{x:.2f}")

comp_tab.to_csv("comparison_table.csv", index=None)

print("Table 3. Explanation of variance in qualitative codes by preferred (k = 15) and alternative (k = 35) LDA topic models.")

comp_tab